# 加载数据

In [1]:
import numpy as np
import matplotlib as plt

In [2]:
from sklearn.datasets import fetch_openml
mnist=fetch_openml('mnist_784',version=1)

# 分割数据

In [3]:
X_train=mnist.data[:60000]
X_test=mnist.data[60000:]
y_train=mnist.target[:60000]
y_test=mnist.target[60000:]

In [4]:
X_train.shape,X_test.shape

((60000, 784), (10000, 784))

# 模型建立

In [5]:
from sklearn.ensemble import RandomForestClassifier
rnd_forest_clf1=RandomForestClassifier(n_estimators=100,random_state=42,n_jobs=-1)

In [6]:
%%time
rnd_forest_clf1.fit(X_train,y_train)

Wall time: 13.9 s


RandomForestClassifier(n_jobs=-1, random_state=42)

# 模型评估

In [7]:
from sklearn.metrics import accuracy_score
y_pred=rnd_forest_clf1.predict(X_test)

In [8]:
accuracy_score(y_test,y_pred)

0.9705

# 数据降维

In [9]:
from sklearn.decomposition import PCA
pca=PCA(n_components=0.95)
X_train_reduced=pca.fit_transform(X_train)

# 模型训练

In [10]:
rnd_forest_clf2=RandomForestClassifier(n_estimators=100,random_state=42,n_jobs=-1)

In [11]:
%%time
rnd_forest_clf2.fit(X_train_reduced,y_train)

Wall time: 32.4 s


RandomForestClassifier(n_jobs=-1, random_state=42)

不好了！ 现在训练实际上慢了两倍多！ 这个怎么可能？ 好吧，正如我们在本章中看到的，降维并不总是会导致更快的训练时间：它取决于数据集、模型和训练算法。 参见图 8-6（上面的 `manifold_decision_boundary_plot*` 图）。 如果您尝试使用 softmax 分类器而不是随机森林分类器，您会发现使用 PCA 时训练时间减少了 3 倍。 实际上，我们将在一秒钟内完成此操作，但首先让我们检查新的随机森林分类器的精度。

# 模型评估

In [12]:
X_test_reduced=pca.transform(X_test)

In [13]:
y_pred=rnd_forest_clf2.predict(X_test_reduced)

In [14]:
accuracy_score(y_pred,y_test)

0.9481

降低维度时，性能略有下降是很常见的，因为我们确实在此过程中丢失了一些有用的信号。 但是，在这种情况下，性能下降相当严重。 所以 PCA 真的没有帮助：它减慢了训练速度并降低了性能。 :(

让我们看看它在使用 softmax 回归时是否有帮助：

# softmax模型建立

In [15]:
from sklearn.linear_model import LogisticRegression

In [18]:
log_clf=LogisticRegression(multi_class='multinomial',solver='lbfgs',random_state=42,n_jobs=-1)

In [19]:
%%time
log_clf.fit(X_train,y_train)

Wall time: 1min 6s


LogisticRegression(multi_class='multinomial', n_jobs=-1, random_state=42)

In [20]:
y_pred=log_clf.predict(X_test)
accuracy_score(y_test,y_pred)

0.9255

In [21]:
log_clf2=LogisticRegression(multi_class='multinomial',solver='lbfgs',random_state=42,n_jobs=-1)

In [22]:
%%time
log_clf2.fit(X_train_reduced,y_train)

Wall time: 15.7 s


LogisticRegression(multi_class='multinomial', n_jobs=-1, random_state=42)

In [24]:
y_pred=log_clf2.predict(X_test_reduced)
accuracy_score(y_test,y_pred)

0.9201

So there you have it: PCA can give you a formidable speedup... but not always!